In [1]:
%matplotlib widget

In [3]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use(['ggplot', 'presentation'])

In [13]:
from numpy import require
from numpy.lib.stride_tricks import as_strided

def get_windowed_view(x, window_length, step_size, ensure_c_contiguity=False):
    """
    Return a moving window view over the data
    Parameters
    ----------
    x : numpy.ndarray
        1- or 2-D array of signals to window. Windows occur along the 0 axis. Must be C-contiguous.
    window_length : int
        Window length/size.
    step_size : int
        Step/stride size for windows - how many samples to step from window center to window center.
    ensure_c_contiguity : bool, optional
        Create a new array with C-contiguity if the passed array is not C-contiguous. This *may* result in the
        memory requirements significantly increasing. Default is False, which will raise a ValueError if `x` is
        not C-contiguous
    Returns
    -------
    x_win : numpy.ndarray
        2- or 3-D array of windows of the original data, of shape (..., L[, ...])
    """
    if not (x.ndim in [1, 2]):
        raise ValueError('Array cannot have more than 2 dimensions.')

    if ensure_c_contiguity:
        x = require(x, requirements=['C'])
    else:
        if not x.flags['C_CONTIGUOUS']:
            raise ValueError("Input array must be C-contiguous.  See numpy.ascontiguousarray")

    if x.ndim == 1:
        nrows = ((x.size - window_length) // step_size) + 1
        n = x.strides[0]
        return as_strided(x, shape=(nrows, window_length), strides=(step_size * n, n), writeable=False)

    else:
        k = x.shape[1]
        nrows = ((x.shape[0] - window_length) // step_size) + 1
        n = x.strides[1]

        new_shape = (nrows, window_length, k)
        new_strides = (step_size * k * n, k * n, n)
        return as_strided(x, shape=new_shape, strides=new_strides, writeable=False)

In [4]:
df3 = pd.read_csv('51888_1020030004.csv', header=None, names=['t', 'x', 'y', 'z'])
acc3 = df3[['x', 'y', 'z']].values
print(1 / np.mean(np.diff(df3['t'])))

197.24258229898155


In [7]:
plt.figure(figsize=(9, 4), constrained_layout=True)
plt.plot(acc3);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
sos = signal.butter(2, 17/100, output='sos')
acc3_f = signal.sosfiltfilt(sos, acc3, axis=0)

plt.figure(figsize=(9, 4), constrained_layout=True)
plt.plot(acc3_f);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
acc3_f_nm = acc3_f - np.mean(acc3_f, axis=0, keepdims=True)

plt.figure(figsize=(9, 4), constrained_layout=True)
plt.plot(acc3_f_nm);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
# window, compute mean (y axis) and standard deviation (3 axis sum)
n_0p1 = int(200 * 0.1)

acc_win = get_windowed_view(acc3_f_nm, n_0p1, 1, True)

vacc_mean = np.mean(acc_win[:, :, 1], axis=1)
acc_ssd = np.sum(np.std(acc_win, axis=1), axis=1)

In [30]:
plt.figure(figsize=(9, 4), constrained_layout=True)
plt.plot(np.arange(acc3.shape[0])[10:-9], vacc_mean, label='Mean');
plt.plot(np.arange(acc3.shape[0])[10:-9], acc_ssd, label='SSD');

plt.axhline(0.77, color='k', linestyle='--')
plt.axhline(0.05, color='k', linestyle='--');

plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …